In [2]:
import pandas as pd
import numpy as np
import joblib
import re
import ast

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from xgboost import XGBRegressor

In [3]:
course_rate = 4.54
cat_imputer = SimpleImputer(strategy='most_frequent')

def get_marka_based_on_model(df, model_to_marka):
    df["Marka_pojazdu"] = df.apply(
        lambda row: model_to_marka.get(row["Model_pojazdu"], row["Marka_pojazdu"])
        if pd.isna(row["Marka_pojazdu"])
        else row["Marka_pojazdu"], axis=1
    )
    return df

def clean_ds(df, df_test):
    cat_source_columns = ["Stan", "Rodzaj_paliwa", "Naped", "Skrzynia_biegow", "Typ_nadwozia", "Kolor", "Kraj_pochodzenia"]
    model_to_marka = df.dropna(subset=["Marka_pojazdu"]).groupby("Model_pojazdu")["Marka_pojazdu"].agg(lambda x: x.mode()[0]).to_dict()
    get_marka_based_on_model(df, model_to_marka)
    get_marka_based_on_model(df_test, model_to_marka)
    df[cat_source_columns] = pd.DataFrame(cat_imputer.fit_transform(df[cat_source_columns]), columns=cat_source_columns) 
    df_test[cat_source_columns] = pd.DataFrame(cat_imputer.transform(df_test[cat_source_columns]), columns=cat_source_columns)
    test_categories = df_test["Rodzaj_paliwa"].unique()
    df = df[df["Rodzaj_paliwa"].isin(test_categories)]
    df = df[~df["Liczba_drzwi"].isin([1, 55])]
    df = df.reset_index(drop=True)
    return df, df_test

In [4]:
scaler = StandardScaler()
imputer = SimpleImputer(strategy='median')

mark_mapper = dict()
model_mapper = dict()
fuel_mapper = dict()
body_mapper = dict()
color_mapper = dict()

def fix_year(date_str):
    if pd.isnull(date_str): 
        return "01/01/2017"
    match = re.search(r"(\d{2})/(\d{2})/(\d{4})", date_str)
    if match:
        day, month, year = match.groups()
        year = int(year)
        if year < 1925 or year > 2021:
            year = 2000 + (year % 100)
        return f"{day}/{month}/{year}"
    return "02/02/2017"

def count_equipment(equipment_str):
    try:
        equipment_list = ast.literal_eval(equipment_str)  
        if isinstance(equipment_list, list):
            return len(equipment_list)  
    except (SyntaxError, ValueError):
        return 0  
    return 0

def create_mapper(df, source_column, result_column, clusters, mapper):
    avg_price = df.groupby(source_column)['Cena'].mean().reset_index()
    kmeans = KMeans(n_clusters=clusters, random_state=42, n_init=5)
    avg_price['cluster'] = kmeans.fit_predict(avg_price[['Cena']])
    avg_price['cluster_str'] = avg_price['cluster'].apply(str)
    mapper = dict(zip(avg_price[source_column], avg_price['cluster_str']))
    df[result_column] = df[source_column].map(mapper)
    return mapper

def prepare_data(df, is_train_data):
    df.loc[df['Stan'] == 'New', 'Przebieg_km'] = 0
    df["generation_set"] = df["Generacja_pojazdu"].apply(lambda x: False if pd.isnull(x) else True)
    df["country_bin"] = df["Kraj_pochodzenia"].apply(lambda x: x in ["Poland", "United States", "Canada"])
    df["owner_bin"] = df["Pierwszy_wlasciciel"].apply(lambda x: False if pd.isnull(x) else x)
    df["equipment_number"] = df["Wyposazenie"].apply(count_equipment)
    df["log_mileage_km"] = np.log1p(df['Przebieg_km'])

    clusters_columns = ["mark_cluster", "model_cluster", "fuel_cluster", "body_cluster", "color_cluster"]
    cat_result_columns = ["Stan", "Naped", "Skrzynia_biegow", "generation_set", "country_bin", "owner_bin"] + clusters_columns 
    numerical_cols = ["log_mileage_km", "Rok_produkcji", "Liczba_drzwi", "Moc_KM", "Pojemnosc_cm3", "Emisja_CO2"]
    # equipment_number adding gave worse results
    if is_train_data:
        df["Fixed_Date"] = df["Data_pierwszej_rejestracji"].apply(fix_year)
        df["Fixed_Date"] = pd.to_datetime(df["Fixed_Date"], dayfirst=True, format='mixed')
        df["Registration_year"] = df["Fixed_Date"].dt.year
        df["Registration_year"] = df["Registration_year"].apply(lambda x: 2017 if x > 2021 or x < 1930 else x)
        X_num = pd.DataFrame(imputer.fit_transform(df[numerical_cols]), columns=numerical_cols)
        X_num = X_num.join(df["Registration_year"])

        global mark_mapper
        mark_mapper = create_mapper(df, "Marka_pojazdu", "mark_cluster", 5, mark_mapper)
        global model_mapper
        model_mapper = create_mapper(df, "Model_pojazdu", "model_cluster", 5, model_mapper)
        global fuel_mapper
        fuel_mapper = create_mapper(df, "Rodzaj_paliwa", "fuel_cluster", 3, fuel_mapper)
        global body_mapper
        body_mapper = create_mapper(df, "Typ_nadwozia", "body_cluster", 3, body_mapper)
        global color_mapper
        color_mapper = create_mapper(df, "Kolor", "color_cluster", 3, color_mapper)
        X_cat = df[cat_result_columns]
        X_encoded = pd.get_dummies(X_cat)
        y = df.Cena
        X = X_num.join(X_encoded)
        return X, y
    X_num = pd.DataFrame(imputer.transform(df[numerical_cols]), columns=numerical_cols)

    df["Fixed_Date"] = df["Data_pierwszej_rejestracji"].apply(fix_year)
    df["Fixed_Date"] = pd.to_datetime(df["Fixed_Date"], dayfirst=True, format='mixed')
    df["Registration_year"] = df["Fixed_Date"].dt.year
    df["Registration_year"] = df["Registration_year"].apply(lambda x: 2021 if x > 2021 else x)
    X_num = X_num.join(df["Registration_year"])

    df["mark_cluster"] = df['Marka_pojazdu'].map(mark_mapper)
    df["model_cluster"] = df['Model_pojazdu'].map(model_mapper)
    df["fuel_cluster"] = df['Rodzaj_paliwa'].map(fuel_mapper)
    df["body_cluster"] = df['Typ_nadwozia'].map(body_mapper)
    df["color_cluster"] = df['Kolor'].map(color_mapper)
    X_test = df[cat_result_columns]
    X_encoded_test = pd.get_dummies(X_test)
    X_test = X_num.join(X_encoded_test)
    return X_test

In [5]:
path = "../data/raw/sales_ads_train.csv"
path_test = "../data/raw/sales_ads_test.csv"
# path_synth = "../data/raw/synthetic_training_data_sdv_pl.csv"
df = pd.read_csv(path)
df_test = pd.read_csv(path_test)
# df_synth = pd.read_csv(path_synth)
# df = pd.concat([df, df_synth])

In [6]:
df, df_test = clean_ds(df, df_test)
X, y = prepare_data(df, True)
X_test = prepare_data(df_test, False)

In [7]:
# pd.DataFrame.to_csv(X, "../data/processed/X.csv", index=False)
# X = pd.read_csv("../data/processed/X.csv")

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
model = XGBRegressor(n_estimators=2300, max_depth=6, eta=0.05, subsample=0.8, colsample_bytree=0.9, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
predictions_df = pd.DataFrame({"Predicted_Price": y_pred}, index=X_val.index)
df_prices = df.merge(predictions_df, left_index=True, right_index=True, how='right')
df_prices["Predicted_Price"] = df_prices.apply(lambda row: row["Predicted_Price"] / course_rate if row["Waluta"] == "EUR" else row["Predicted_Price"], axis=1)
y_pred = df_prices["Predicted_Price"].values
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.4f}")
errors = np.abs(y_val - y_pred)
df_errors = pd.DataFrame({
    'y_val': y_val / 1000,
    'y_pred': y_pred / 1000,
    'error': errors / 1000,
})
df_errors_sorted = df_errors.sort_values(by='error', ascending=False)
df_errors_sorted.head(5)

Root Mean Squared Error (RMSE): 21448.70
R² Score: 0.9366


,y_val,y_pred,error
21921,1388.000,362.313031,1025.686969
39570,2370.000,1919.891875,450.108125
69119,660.000,246.302141,413.697859
113217,63.900,457.100125,393.200125
29368,699.999,327.209281,372.789719


Root Mean Squared Error (RMSE): 21448.70
R² Score: 0.9366

In [9]:
is_grid_search = False
if is_grid_search:
    param_grid = {
        'n_estimators': [1000, 1200, 800],
        'max_depth': [5, 7, 9],
        'eta': [0.05, 1],
        'subsample': [0.7, 0.9],
        'colsample_bytree': [0.8, 1],
    }

    model = XGBRegressor()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=8, verbose=1, n_jobs=-1)
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    print("Best score: ", -grid_search.best_score_)
    print("Best Parameters found: ", best_params)

In [11]:
is_final_predict = True

if is_final_predict:
    # model = RandomForestRegressor(n_estimators=100, random_state=42)
    model = XGBRegressor(n_estimators=2500, max_depth=5, eta=0.05, subsample=0.8, colsample_bytree=0.9, random_state=42)
    model.fit(X, y)
    joblib.dump(model, "../models/xgboost.joblib")
    y_pred_init = model.predict(X_test)
    df_test["Predicted_Price"] = pd.DataFrame({"Predicted_Price": y_pred_init}, index=df_test.index)
    df_test["Predicted_Price"] = df_test.apply(lambda row: row["Predicted_Price"] / course_rate if row["Waluta"] == "EUR" else row["Predicted_Price"], axis=1)
    y_pred_test = df_test["Predicted_Price"].values

    df_output = pd.DataFrame({
        'ID': np.arange(1, len(y_pred_test) + 1),  
        'Cena': y_pred_test
    })

    df_output.to_csv("../results/xgboost.csv", index=False)